In [1]:
import folium
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import os

In [2]:
METADATA_PATH = '../data/metadata/train_val/zurich/'

## Load Dataframes

In [3]:
DATA_PATH_QUERY = os.path.join(METADATA_PATH, 'query/raw.csv')
DATA_PATH_DATABASE = os.path.join(METADATA_PATH, 'database/raw.csv')

In [4]:
df = pd.read_csv(DATA_PATH_DATABASE)
dfq = pd.read_csv(DATA_PATH_QUERY)

## Inspect a City

In this code block, we will inspect the routes in the dataset. 

Different map types can be found on [Github](https://python-visualization.github.io/folium/modules.html)

In [5]:
MAP_TYPE = 'Stamen Terrain'

In [6]:
def Haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 * 10**3 #Radius of earth in meters.
    return c * r

In [7]:
m = folium.Map(
    location=[df['lat'][0], df['lon'][0]],
    zoom_start=12,
    tiles=MAP_TYPE
)

In [8]:
def SplitCoordintesByFrame(coordinates, frames):
    previous_split = 0
    for i in range(1, len(frames)):
        if frames[i-1][0] + 1 != frames[i][0]:
            route_seq = coordinates[previous_split:i]
            folium.PolyLine(route_seq, weight=5.5).add_to(m)
            previous_split = i

In [103]:

# This is another way of generating the exact routes taken, but it is not super robust i.e. it seems to throw away some data. 

"""
DATA_PATH_DATABASE = os.path.join(METADATA_PATH, 'database/seq_info.csv')
df_seq = pd.read_csv(DATA_PATH_DATABASE)
df_full = pd.merge(df, df_seq[["key", "sequence_key", "frame_number"]], on='key')

for sequence_key in df_full["sequence_key"].unique():
    coordinates = df_full.loc[df_full["sequence_key"] == sequence_key][["lat", "lon", "frame_number"]]
    coordinates_list = coordinates[["lat", "lon"]].values.tolist()
    frames_list = coordinates[["frame_number"]].values.tolist()
    SplitCoordintesByFrame(coordinates_list, frames_list)
"""

'\nDATA_PATH_DATABASE = os.path.join(METADATA_PATH, \'database/seq_info.csv\')\ndf_seq = pd.read_csv(DATA_PATH_DATABASE)\ndf_full = pd.merge(df, df_seq[["key", "sequence_key", "frame_number"]], on=\'key\')\n\nfor sequence_key in df_full["sequence_key"].unique():\n    coordinates = df_full.loc[df_full["sequence_key"] == sequence_key][["lat", "lon", "frame_number"]]\n    coordinates_list = coordinates[["lat", "lon"]].values.tolist()\n    frames_list = coordinates[["frame_number"]].values.tolist()\n    SplitCoordintesByFrame(coordinates_list, frames_list)\n'

In [9]:
point = 0,0
points = []
for i in range(0, len(df)):
    if Haversine(point[1],point[0], df['lon'][i], df['lat'][i]) > 25: # 25 Meters (~82 feet)
        if len(points) > 0:
            folium.PolyLine(points, weight=5.5).add_to(m)
        point = (df['lat'][i], df['lon'][i])
        points = [point]
    else:
        point = (df['lat'][i], df['lon'][i])
        points.append(point)

In [10]:
m

### Inspect City by Season

In [88]:
m = folium.Map(
    location=[df['lat'][0], df['lon'][0]],
    zoom_start=12,
    tiles=MAP_TYPE
)

In [89]:
SeasonsByMonth = [(month%12 + 3)//3 for month in range(1, 13)]
def DateToYear(Date):
    Month = int(Date.split('-')[1]) - 1
    return SeasonsByMonth[Month]

MonthToColor = {
    1 : 'Blue',
    2 : 'Yellow',
    3 : 'Green',
    4 : 'Red'
}


In [90]:
WinterGroup = folium.FeatureGroup(name='Winter')
SpringGroup = folium.FeatureGroup(name='Spring')
SummerGroup = folium.FeatureGroup(name='Summer')
FallGroup = folium.FeatureGroup(name='Fall')
MonthToGroup = [WinterGroup, SpringGroup, SummerGroup, FallGroup]

In [91]:
point = 0,0
points = []
for i in range(0, len(df)):
    if Haversine(point[1],point[0], df['lon'][i], df['lat'][i]) > 25: # 25 Meters (~82 feet)
        if len(points) > 0:
            SeasonId = DateToYear(df['captured_at'][i])
            LineColor = MonthToColor[SeasonId]
            SeasonGroup = MonthToGroup[SeasonId - 1]
            folium.PolyLine(points, weight=5.5, color=LineColor).add_to(SeasonGroup)
        point = (df['lat'][i], df['lon'][i])
        points = [point]
    else:
        point = (df['lat'][i], df['lon'][i])
        points.append(point)

for FeatureGroup in MonthToGroup:
    FeatureGroup.add_to(m)

folium.map.LayerControl().add_to(m)

In [92]:
m

## Database and Query 

In [93]:
m = folium.Map(
    location=[df['lat'][0], df['lon'][0]],
    zoom_start=12,
    tiles=MAP_TYPE
)

DBGroup = folium.FeatureGroup(name='Database')
QueryGroup = folium.FeatureGroup(name='Query')

In [94]:
point = 0,0
points = []
for i in range(0, len(df)):
    if Haversine(point[1],point[0], df['lon'][i], df['lat'][i]) > 25: # 25 Meters (~82 feet)
        if len(points) > 0:
            folium.PolyLine(points, weight=5.5, color='Red').add_to(DBGroup)
        point = (df['lat'][i], df['lon'][i])
        points = [point]
    else:
        point = (df['lat'][i], df['lon'][i])
        points.append(point)

point = 0,0
points = []
for i in range(0, len(dfq)):
    if Haversine(point[1],point[0], dfq['lon'][i], dfq['lat'][i]) > 25:
        if len(points) > 0:
            folium.PolyLine(points, weight=5.5, color='Blue').add_to(QueryGroup) # 25 Meters (~82 feet)
        point = (dfq['lat'][i], dfq['lon'][i])
        points = [point]
    else:
        point = (dfq['lat'][i], dfq['lon'][i])
        points.append(point)

In [95]:
DBGroup.add_to(m)
QueryGroup.add_to(m)
folium.map.LayerControl().add_to(m)

m.save('DBAndQuery.html')
m

## Inspect City with Images

In [11]:
m = folium.Map(location=[df['lat'][0], df['lon'][0]], zoom_start=12, tiles=MAP_TYPE)

In [12]:
def PlotPointWithImage(Latitude, Longitude, ImageKey):
    html = "<img src='/Users/alexanderholstrup/git/VPRLossFunction/data/data/train_val/trondheim/database/images/{}.jpg'>".format(ImageKey)
    
    marker = folium.Marker(
        location=[Latitude, Longitude],
        radius=4,
        popup=html
    )
    marker.add_to(m)

In [13]:
for i in range(0, 100, 2):
    PlotPointWithImage(df['lat'][i], df['lon'][i], df['key'][i])

In [14]:
m.save('MapWithImages.html')

In [17]:
df['key'] == 1

0       False
1       False
2       False
3       False
4       False
        ...  
2986    False
2987    False
2988    False
2989    False
2990    False
Name: key, Length: 2991, dtype: bool

In [32]:
df.loc[df['key'] == 'vOAkkwNtv5Fcy3jY6jI4LA']

,Unnamed: 0,key,lon,lat,ca,captured_at,pano
1631,1631,vOAkkwNtv5Fcy3jY6jI4LA,8.439736,47.417623,294.73517,2000-02-07,False


In [89]:
p = dfq.loc[dfq['key'] == 'PkvDVIeXmLMdCcHGMtpJxQ'] 
p['lon']

31    8.534346
Name: lon, dtype: float64

In [87]:
df.loc[df['key'] == 'PkvDVIeXmLMdCcHGMtpJxQ']

,Unnamed: 0,key,lon,lat,ca,captured_at,pano


In [23]:
Haversine(8.587179, 47.408761, 8.599693, 47.406018)

989.8974172455488

In [137]:
def PlotTuple(tuple):
    query = tuple[0]
    positive = tuple[1]
    negative = tuple[2]
    for n in negative:
        PlotPointWithImage(n, 'N')
    PlotPointWithImage(query, 'Q')
    PlotPointWithImage(positive, 'P')


def PlotPointWithImage(ImageKey, label):
    if 'database' in ImageKey:
        html = "<img src='/Users/alexanderholstrup/git/VPRLossFunction/data/data/train_val/zurich/database/images/{}.jpg'>".format(ImageKey)
        dataframe = df
        key = ImageKey.split('/')[-1][:-4]
    else:
        html = "<img src='/Users/alexanderholstrup/git/VPRLossFunction/data/data/train_val/zurich/query/images/{}.jpg'>".format(ImageKey)
        dataframe = dfq
        key = ImageKey.split('/')[-1][:-4]
    
    point = dataframe.loc[dataframe['key'] == key]
    Latitude = point['lat']
    Longitude = point['lon']

    
    marker = folium.Marker(
        location=[Latitude, Longitude],
        radius=4,
        popup=label #html,
    )
    marker.add_to(m)


In [138]:
m = folium.Map(location=[df['lat'][0], df['lon'][0]], zoom_start=12, tiles=MAP_TYPE)
#sample = ['data/mapillary/train_val/zurich/query/images/PkvDVIeXmLMdCcHGMtpJxQ.jpg', 'data/mapillary/train_val/zurich/database/images/eg2j48yMmfIYAOBvXKbiuA.jpg', ['data/mapillary/train_val/zurich/query/images/PkvDVIeXmLMdCcHGMtpJxQ.jpg', 'data/mapillary/train_val/zurich/query/images/Pm3Ozp-eiDm044yr5Uk3Dg.jpg', 'data/mapillary/train_val/zurich/query/images/tSh7CualX4sfq_GNUCCIXQ.jpg', 'data/mapillary/train_val/zurich/query/images/cZmPWAGhrFXGAF43yeKm1w.jpg', 'data/mapillary/train_val/zurich/query/images/4uyKIMREtWgtyxV7F4jnyw.jpg']]

#sample = ['data/mapillary/train_val/zurich/query/images/DJn8mOgddYc7239Ze1-xtk.jpg', 'data/mapillary/train_val/zurich/database/images/HhFa9T04mt0YMzvDMn-cFQ.jpg', ['data/mapillary/train_val/zurich/query/images/DJn8mOgddYc7239Ze1-xtk.jpg', 'data/mapillary/train_val/zurich/query/images/tBxVRXJbNcaUeHnRET4fca.jpg', 'data/mapillary/train_val/zurich/query/images/l4OJMyhVomS2253juPcqB-.jpg', 'data/mapillary/train_val/zurich/query/images/egwIb9Eoep8V28oudYlNce.jpg', 'data/mapillary/train_val/zurich/query/images/GpNdf_iHAfvEP6lYofuMbL.jpg']]

sample = ['data/mapillary/train_val/zurich/query/images/XDqcwrhp9V8tDteRpmHudw.jpg', 'data/mapillary/train_val/zurich/database/images/zobdL2eZeGp0nfY68AbkCA.jpg', ['data/mapillary/train_val/zurich/query/images/XDqcwrhp9V8tDteRpmHudw.jpg', 'data/mapillary/train_val/zurich/query/images/-D3dDkhM5rjIJhj1SPdroA.jpg', 'data/mapillary/train_val/zurich/query/images/F4Z-K4ZqSPDXXHQT_bPhKw.jpg', 'data/mapillary/train_val/zurich/query/images/-fOHAPAq-KX4XPlg0JJPvg.jpg', 'data/mapillary/train_val/zurich/query/images/j5QDXJ00JsGza_UNWT3dwA.jpg']]

PlotTuple(sample)

In [139]:
m